## Sift Evaluator

The objectives of this notebook are to:

- Define functions that query the data based on different parameters (distance metric, transformations?)
- Define functions to evaluate the truth of each returned query parameter
- Define functions to calculate mAP and precision@k for the above output
- Investigate the effects of different parameters on mAP, p@k.

____

In [32]:
import numpy as np
import unittest
import sklearn.metrics.pairwise

____
## Query Functions

In [ ]:
def basic_query(query_image, query_target):
    """Return the indexes of the query_target images, arranged in ascending euclidean distance as compared to the query image"""
    
    # copy pasted code to investigate
    query = query_image.reshape((1, -1))
    D = sklearn.metrics.pairwise.euclidean_distances(query_target, query).squeeze()
    index = np.argsort(D)

    return(index)

_____
## Query to Truth Value Functions

In [41]:
def query_results_to_truth_values(query_image_building, query_results, image_names):
    # syntax??
    #results = [True if query_image_building == image_names[query_result] for query_result in query_results; else False]
    pass

_______
## Truth Value Metrics Functions

In [38]:
def precision_at_k(truth_values, k, warnings=True):
    """Return proportions of true values in the first k elements.
    If warnings=True and all true values occur before the kth element, raise an error and print the value at k and the value at the last true value."""
    p_at_k = truth_values[:k].count(True) / k

    ## Complete code to output p at k, p at last value
    if warnings:
        if k < len(truth_values):
            if truth_values[k:].count(True) == 0:
                raise ValueError("All true values are before the first k values")
    
    return(p_at_k)


## Create a more computationally efficient version using true positives, false positives?
def average_precision(truth_values):
    """Given a boolean input of whether returned query values are correct or false, return the average precision.
    e.g. average_precision([True, True, False, True]) ~ 0.85
    """
    precisions = []
    for (index, val) in enumerate(truth_values):
        if val: # == True
            precisions.append(truth_values[:index + 1].count(True) / (index + 1))      

    return(np.mean(precisions))

#tbd
def mean_average_precision():
    pass

In [39]:
import unittest

class TestMetrics(unittest.TestCase):

    def test_precision_at_k(self):
        self.assertEqual(precision_at_k([True,False,True, True],4), 0.75)
        self.assertEqual(precision_at_k([True, False, True, False, True], 2), 0.5)

    def test_averageprecision(self):
        self.assertAlmostEqual(average_precision([True, False, True, True]), np.mean([1,2/3,3/4]))
        self.assertAlmostEqual(average_precision([False, False, True, False, True]), np.mean([1/3,2/5]))
    
    # def test_meanaverageprecision(self)

unittest.main(argv=[''], verbosity=2, exit=False)

test_averageprecision (__main__.TestMetrics) ... ok
test_precision_at_k (__main__.TestMetrics) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
